![NYC Logo](https://www1.nyc.gov/assets/tlc/images/content/pages/home/nyc-tlc-logo.png)

# Convert CSV's with NYC Taxi Data to Parquet

> **! NOTICE !** In May, 2022, the NYC Taxi Commission changed the format they use to share their data.  Previously, the data was shared in CSV format.
Now it's shared as Parquet files.  This is the ***old*** code that converted the CSV files.  Don't use this code.  It's only here for historical
reference.

This notebook uses Spark to read the NYC Taxi Data CSV files and convert all of the data into a Parquet file.  (*Technically*, it's creating a Databricks Delta file, but that is just
regular ol' Parquet with some additional sidecar data.)

We want to combine all of the CSV's into one, master dataset.  However, the schemas for the CSV vary between taxi types and change over time within the taxi type.  Therefore, we
must devote some effort to standardizing the schemas to one, common schema.

First, specify the location of the CSV files (the input for this process) and the location where we will store the Parquet output.

In [0]:
csv_path = "/mnt/taxi/csv"
parquet_path = "/mnt/taxi/parquet"

Next, we create a Delta table with the schema we ultimately want for our dataset.  This schema was designed by analyzing the available columns in all of the CSV files and determining all of the possible data points we need to capture.

In [0]:
spark.sql("DROP TABLE IF EXISTS taxi")

sql_statement = '''
  CREATE TABLE taxi (
    taxi_type string NOT NULL,
    year int NOT NULL,
    filename string NOT NULL,
    dispatching_base_num string,
    vendor_id string,
    vendor_name string,
    pickup_datetime timestamp,
    pickup_location_id string,
    pickup_latitude decimal(9, 5),
    pickup_longitude decimal(9, 5),
    dropoff_datetime timestamp,
    dropoff_location_id string,
    dropoff_latitude decimal(9, 5),
    dropoff_longitude decimal(9, 5),
    passenger_count integer,
    trip_distance decimal(9, 5),
    trip_type string,
    payment_type string,
    rate_code string,
    shared_ride string,
    store_and_forward string,
    fare_amount decimal(8, 2),
    extra decimal(8, 2),
    mta_tax decimal(8, 2),
    tip_amount decimal(8, 2),
    tolls_amount decimal(8, 2),
    surcharge decimal(8, 2),
    ehail_fee decimal(8, 2),
    total_amount decimal(8, 2)
  )
  USING DELTA
  PARTITIONED BY (taxi_type, year)
  LOCATION '{}'
'''.format(parquet_path)

spark.sql(sql_statement)

Out[ 2 ]: DataFrame[]

Here's a little function that will recursively search subdirectories in the DBFS and return all of the files in the directory tree.

In [0]:
def getFiles(path):
  files = dbutils.fs.ls(path)
  output = []
  
  for f in files:
    if f.name.endswith("/"):
      output.extend(getFiles(f.path))
    else:
      output.append(f)
  
  return(output)

Now we do the real work!

1. Recursively get all of the CSV files from the input location
2. Loop through all of the files and read each file in as a Spark dataframe.  Don't apply any schema; just read whatever columns we find as strings
3. We add new columns to each dataframe containing the taxi type and the name of the CSV file from which the data was read
4. Some of the CSV files have leading spaces in the column names.  We rename those columns to remove those spaces and to convert all column names to lowercase (for consistency)
5. The names of the columns can vary between files.  So we've hard-coded a mapping to convert column names to a standard name.
6. For any columns with a name ending in "datetime", parse the string contents of the column into a timestamp
7. Add a new column by extracting the year from the `pickup_datetime` column
8. Compare the schema of our dataframe to the target Delta table.  If there are any columns missing from our dataframe, add them with `null` values.
9. Loop through all of the columns in our dataframe.  If a column's datatype doesn't match the datatype in the target Delta table, cast the column to the appropriate datatype.
10. Reorder the columns in our dataframe to match the order of columns in the target Delta table.
11. At last!  We insert the dataframe into the Delta table.  This writes the data out as Parquet in our target location.
12. Go back to Step 2 and process the next CSV file.

In [0]:
import pyspark.sql.functions as pyf
from pyspark.sql import Row

all_files = getFiles(csv_path)

target_cols = sqlContext.table("taxi").dtypes

# Loop through all of the CSV files
for f in all_files:
  print("Processing file {}".format(f))
  
  taxi_type = f.path.split('/')[-2]
  
  # Use Spark to read the current CSV file into a dataframe and add some new columns
  raw_df = spark\
            .read.option("header", "true")\
            .csv(f.path)\
            .withColumn("filename",  pyf.input_file_name())\
            .withColumn("taxi_type", pyf.lit(taxi_type))
  
  # Remove spaces from column names and convert to lowercase
  for n in raw_df.schema.names:
    clean_name = n.strip().lower()
    if n != clean_name:
      raw_df = raw_df.withColumnRenamed(n, clean_name)
  
  # Convert columns to standard names
  if "dolocationid"          in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("dolocationid", "dropoff_location_id")
  if "end_lat"               in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("end_lat", "dropoff_latitude")
  if "end_lon"               in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("end_lon", "dropoff_longitude")
  if "fare_amt"              in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("fare_amt", "fare_amount")
  if "improvement_surcharge" in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("improvement_surcharge", "surcharge")
  if "locationid"            in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("locationid", "pickup_location_id")
  if "lpep_dropoff_datetime" in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("lpep_dropoff_datetime", "dropoff_datetime")
  if "lpep_pickup_datetime"  in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("lpep_pickup_datetime", "pickup_datetime")
  if "pickup_date"           in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("pickup_date", "pickup_datetime")
  if "pulocationid"          in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("pulocationid", "pickup_location_id")
  if "ratecodeid"            in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("ratecodeid", "rate_code")
  if "sr_flag"               in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("sr_flag", "shared_ride")
  if "start_lat"             in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("start_lat", "pickup_latitude")
  if "start_lon"             in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("start_lon", "pickup_longitude")
  if "store_and_fwd_flag"    in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("store_and_fwd_flag", "store_and_forward")
  if "tip_amt"               in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("tip_amt", "tip_amount")
  if "tolls_amt"             in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("tolls_amt", "tolls_amount")
  if "total_amt"             in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("total_amt", "total_amount")
  if "tpep_dropoff_datetime" in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("tpep_dropoff_datetime", "dropoff_datetime")
  if "tpep_pickup_datetime"  in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("tpep_pickup_datetime", "pickup_datetime")
  if "trip_dropoff_datetime" in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("trip_dropoff_datetime", "dropoff_datetime")
  if "trip_pickup_datetime"  in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("trip_pickup_datetime", "pickup_datetime")
  if "vendorid"              in raw_df.schema.names: raw_df = raw_df.withColumnRenamed("vendorid", "vendor_id")

  # Parse datetime columns to a timestamp
  for dt in raw_df.dtypes:
    if dt[0].endswith("datetime"):
      raw_df = raw_df.withColumn(dt[0], pyf.to_timestamp(dt[0], 'yyyy-MM-dd HH:mm:ss'))
  
  # Add a "year" column
  raw_df = raw_df.withColumn("year", pyf.year("pickup_datetime"))
  
  # Add missing columns
  for tc in target_cols:
    if tc[0] not in raw_df.schema.names:
      raw_df = raw_df.withColumn(tc[0], pyf.lit(None))

  # Make columns datatype match target
  for tc in target_cols:
    ac = next(x for x in raw_df.dtypes if x[0] == tc[0])
    if ac[1] != tc[1]:
      raw_df = raw_df.withColumn(ac[0], raw_df[ac[0]].cast(tc[1]))

  # Reorder columns to match target
  raw_df = raw_df.select([x[0] for x in target_cols])

  raw_df.write.insertInto("taxi", overwrite = False)

Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-01.csv', name='2015-01.csv', size=85733063)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-02.csv', name='2015-02.csv', size=97863482)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-03.csv', name='2015-03.csv', size=102220197)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-04.csv', name='2015-04.csv', size=121250461)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-05.csv', name='2015-05.csv', size=133469666)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-06.csv', name='2015-06.csv', size=132209226)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-07.csv', name='2015-07.csv', size=137153004)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-08.csv', name='2015-08.csv', size=164291700)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-09.csv', name='2015-09.csv', size=205607912)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-10.csv', name='2015-10.csv', size=267154747)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-11.csv', name='2015-11.csv', size=259214640)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2015-12.csv', name='2015-12.csv', size=271987599)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-01.csv', name='2016-01.csv', size=269366796)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-02.csv', name='2016-02.csv', size=292747091)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-03.csv', name='2016-03.csv', size=299642308)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-04.csv', name='2016-04.csv', size=315566565)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-05.csv', name='2016-05.csv', size=334436403)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-06.csv', name='2016-06.csv', size=334753407)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-07.csv', name='2016-07.csv', size=343027537)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-08.csv', name='2016-08.csv', size=348771009)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-09.csv', name='2016-09.csv', size=364401686)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-10.csv', name='2016-10.csv', size=392966473)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-11.csv', name='2016-11.csv', size=386130495)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2016-12.csv', name='2016-12.csv', size=430102704)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-01.csv', name='2017-01.csv', size=449961921)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-02.csv', name='2017-02.csv', size=437760457)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-03.csv', name='2017-03.csv', size=517512752)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-04.csv', name='2017-04.csv', size=486596109)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-05.csv', name='2017-05.csv', size=509232843)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-06.csv', name='2017-06.csv', size=836353639)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-07.csv', name='2017-07.csv', size=1060291820)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-08.csv', name='2017-08.csv', size=1076708571)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-09.csv', name='2017-09.csv', size=1125198924)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-10.csv', name='2017-10.csv', size=1211831382)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-11.csv', name='2017-11.csv', size=1223139182)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2017-12.csv', name='2017-12.csv', size=1353700482)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2018-01.csv', name='2018-01.csv', size=1342303302)
Processing file FileInfo(path='dbfs:/mnt/taxi/csv/fhv/2018-02.csv', name='2018-02.csv', size=1310711500)
Processing file FileInfo(path='dbfs:/mnt/t

Because we loaded the CSV data into our table with multiple writes, we will have many small Parquet files.  Use Databricks Delta functionality to clean up those files and optimize
the file layout.

In [0]:
spark.sql("OPTIMIZE taxi ZORDER BY (pickup_datetime)")

spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", 'false')
spark.sql("VACUUM taxi RETAIN 0 hours")
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", 'true')

We're all done!  Now we just need to check our work to make sure everything looks good.

First, read all of the CSV files at once and count the rows in them.  This will let us check below to make sure we got all of the data.

In [0]:
csv_df = spark.read.option("header", "true").csv("{}/*/*.csv".format(csv_path))
"{:,}".format(csv_df.count())

Out[ 5 ]: '2,061,783,136'

Now create a nice matrix that shows the number of rows by `year` and `taxi_data`.  Add a column that shows the total rows for each year and a summary row at the bottom that shows the
number of rows for each taxi type.  The grand total in the bottom-right cell should match the total number of rows we read from CSV in the command above.

In [0]:
import pyspark.sql.functions as pyf

# Read the data that we read as parquet (even though it's actually a Delta table)
spark.conf.set("spark.databricks.delta.formatCheck.enabled", "false")
parq_df = spark.read.parquet(parquet_path)
spark.conf.set("spark.databricks.delta.formatCheck.enabled", "true")

# Summarize the row counts by year and taxi_type
summary_df = parq_df\
              .withColumn("year", pyf.col("year").cast("string"))\
              .groupBy("year")\
              .pivot("taxi_type")\
              .count()\
              .fillna(0)\
              .orderBy("year")

# Add a "total" column that shows the total counts for each year
summary_df = summary_df.withColumn('total', sum(summary_df[col] for col in summary_df.columns if col != "year"))

# Add a summary row that shows the total counts for each column
exprs = [pyf.sum(x).alias(x) for x in summary_df.columns if x != "year"]
summary_row = summary_df.agg(*exprs).selectExpr("'total' AS year", "*")
summary_df = summary_df.union(summary_row)

display(summary_df)

year,fhv,green,yellow,total
2001,0,0,10,10
2002,0,0,16,16
2003,0,0,16,16
2008,0,55,188,243
2009,0,109,170896408,170896517
2010,0,210,169001153,169001363
2011,0,0,176897208,176897208
2012,0,3,178544324,178544327
2013,0,1210811,173179759,174390570
2014,0,15837001,165114361,180951362


Just for fun, add a visualization that shows the rowcounts by year.  This lets us see that yellow taxi rides are declining over time while FHV rides are exploding.

In [0]:
display(
  parq_df\
    .filter("year BETWEEN 2009 AND 2018")\
    .groupBy("year")\
    .pivot("taxi_type")\
    .count()\
    .orderBy("year")
)

year,fhv,green,yellow
2009,null,109,170896408
2010,null,210,169001153
2011,null,null,176897208
2012,null,3,178544324
2013,null,1210811,173179759
2014,null,15837001,165114361
2015,63867609,19233765,146112989
2016,133285141,16385532,131165043
2017,192280677,11740514,113496932
2018,124871124,4737082,53925040
